This notebook will introduce new Python and Astropy users to various aspects of the Python astronomy workflow that may help them to take better advantage of the rest of the STAK notebooks. If you are a new Python or Astropy user, we highly recommend reading through all sections of this notebook. We will discuss:

* [Workflow Philosophy, the difference in workflow between IRAF and Python](#workflow).
* [FITS File I/O, opening and updating FITS files in Python](#fits).
* [IRAF/IDL to Python gotchas](#gotchas).
* [Helpful links and additional resources](#links).

<a id='workflow'></a>

# Workflow Philosophy

The traditional IRAF workflow consists of individual tasks that take a FITS file as input, run a data processing algorithm on the input file, and return an updated output file for the user.  The workflow in Python is very different.  Like most interpreted languages, data manipulation is more a flow of operations expressed in regular code than individual tasks that operate on files.

For example, instead of providing an input FITS file to a task and getting an updated output file, for many astronomy Python functions the user will open the FITS file themselves, access the file object, and continue with any analysis or data processing from there.  Once the data manipulation is done (which could be a long sequence of manipulations), if the user would like to update their FITS files they actively save the changes to the file.  This could include saving out intermediate steps to new files, if the user so desires. While this may take a few more lines of code to open and close files, you gain the major advantage of flexibility, customization and freedom, as well as potentially more efficient I/O.  Often times the increase in the number of lines of code allows the user to be more explicit about what the code is doing, instead of relying on complex parameter files and black box code.  If a user finds themselves running the same code numerous time and would find a one line call useful, they can always put their workflow into a function in a Python file and it will be ready to import and use in any local Python session.

Particularly useful for this kind of scientific workflow is the [Jupyter Notebooks](https://jupyter-notebook.readthedocs.io/en/stable/). This tool allows users to run fully interactive Python sessions with the ability to save the code that was run, it's screen output, the order it was run in, and other helpful tools.  If you are new to Python, we highly recommend taking advantage of this tool.  All STAK notebooks were written in Jupyter Notebooks.  This means the user can downloads these notebooks and run all code examples locally, adding additional code to use their data in the provided examples. For a more in-depth example of how to effectively use Python and the Jupyter Notebooks for data analysis, there are some good video tutorials by [Jake VanderPlas here](https://jakevdp.github.io/blog/2017/03/03/reproducible-data-analysis-in-jupyter/).

<a id='fits'></a>

# FITS File I/O

Utilities to open, edit, and save FITS files are contained in the [Astropy package](http://docs.astropy.org/en/stable/). Although there are many documentation resources for this package (which we will reference below in the [links and resources section](#links)), it can be a bit overwhelming for new users. To help new users adjust to a more Pythonic workflow, some of the examples shown in the STAK notebooks do not show the opening and closing of the FITS file.  Instead, we show the basic calls to do this below.

Before we show any code, we will go over a basic explanation of the Astropy classes and object types for FITS files. The highest level class for a FITS file is the [HDUList](http://docs.astropy.org/en/stable/io/fits/api/hdulists.html).  This is the object type you get when you first open a FITS file in Astropy.  The HDUList contains a sequence of HDU (Header Data Unit) objects. There are three common flavors of HDU object, a [PrimaryHDU](http://docs.astropy.org/en/stable/io/fits/api/hdus.html#), an [ImageHDU](http://docs.astropy.org/en/stable/io/fits/api/images.html#astropy.io.fits.ImageHDU) and a [BinTableHDU](http://docs.astropy.org/en/stable/io/fits/api/tables.html#astropy.io.fits.BinTableHDU).  As you would expect, the PrimaryHDU object contains the primary header, the ImageHDU object contains an image array and it's associated header, and a BinTableHDU object contains a binary table and it's associated header.

The next level of objects are the individual header and data objects.  Headers are stored in a [Header](http://docs.astropy.org/en/stable/io/fits/api/headers.html) object.  Images are stored in a [Numpy array](https://docs.scipy.org/doc/numpy/reference/generated/numpy.array.html) object. Binary tables are stored in a [record array](http://docs.astropy.org/en/stable/io/fits/usage/table.html).

Now that we've covered the object types, we will show with code snippets how to extract and work with these objects using a FITS file containing image data. For more information on table data see the [Astropy FITS Table tutorial](http://www.astropy.org/astropy-tutorials/FITS-tables.html). For more detailed coverage of the avaialbe tools in the Astropy FITS module see the [narrative documentation page](http://docs.astropy.org/en/stable/io/fits/).

In [6]:
from astropy.io import fits

In [7]:
# Assign filename to the variable test_file
test_file = '/eng/ssb/iraf_transition/test_data/wfc3data_flt.fits'

Below we will open the FITS file. You can open the file in various modes, for this example we will open in update mode.  The default mode is read only.

In [8]:
# Open the FITS file with Astropy
HDUList_object = fits.open(test_file, mode='update')

Next we will show the info print out for this HDUList object using the info() method. Notice the No. and Type columns.  These will be useful for indexing the HDUList.

In [9]:
# HDUList info call
HDUList_object.info()

Filename: /eng/ssb/iraf_transition/test_data/wfc3data_flt.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     265   ()      
  1  SCI           1 ImageHDU       140   (1014, 1014)   float32   
  2  ERR           1 ImageHDU        51   (1014, 1014)   float32   
  3  DQ            1 ImageHDU        43   (1014, 1014)   int16   
  4  SAMP          1 ImageHDU        37   (1014, 1014)   int16   
  5  TIME          1 ImageHDU        37   (1014, 1014)   float32   
  6  WCSCORR       1 BinTableHDU     59   7R x 24C   [40A, I, A, 24A, 24A, 24A, 24A, D, D, D, D, D, D, D, D, 24A, 24A, D, D, D, D, J, 40A, 128A]   


Now we will extract the primary header into the variable `primary_header`

In [10]:
# Extract primary header
primary_header = HDUList_object[0].header

# Index header object with keyword name and print value
print(primary_header['FILENAME'])

iczgs3y5q_flt.fits


Next we extract the image data into a variable called `image_data` from the first image extension. We will index this using the index number from the No. column returned by `info()`.  This variable is a numpy array object and the object that allows you to directly interact with the image data. For more information on indexing here is a useful [Numpy documentation page](https://docs.scipy.org/doc/numpy/user/basics.indexing.html).

In [12]:
# Extract image data from the first extension
image_data = HDUList_object[1].data
print(image_data)

[[  0.86692303   0.80678135   0.78075016 ...,   4.54734516  10.16776085
    5.09605217]
 [  0.83312052   0.76854318   0.88651454 ...,   0.81159276   5.15299034
   -0.40152752]
 [  0.77341086   0.80276382   0.78382087 ...,   4.50097942 -11.52676868
    3.39691401]
 ..., 
 [ 48.13546371   6.93788576 -22.45811272 ...,  22.81965828  28.73524284
   34.41676331]
 [ 44.60122681 -10.17064476  -9.5944128  ...,  14.01812649  29.81622887
    5.58320761]
 [ 54.62176895  30.16882896  79.96840668 ...,   0.          10.12944221
   24.66659927]]


We now have two options for saving out the FITS information.  

We can save it out to the original file by using our `HDUList` file object and the `close` argument. If the file was opened using the update mode, this will flush (write) the file changes.  If the file was opened in the default readonly mode, it will **not** be updated when closed. 

We can also use the `writeto` method to save the `HDUObject` to a new file.  

`writeto` will also save to the original file if you provide it with the original filename, and use the `overwrite=True` parameter.

**No matter which mode you used to open the FITS file, you should still call the close method to close the open FITS file, writeto does not close the file.**

In [17]:
# Save using the writeto method to a new file
HDUList_object.writeto("acsdata_new.fits")

# Save using the writeto method, overwriting the original file
HDUList_object.writeto("/eng/ssb/iraf_transition/test_data/acsdata_flt.fits", overwrite=True)

In [5]:
# Save to same file using close
# We show this last because we need to close the file, even after using a writeto
HDUList_object.close()

<a id='gotchas'></a>

# IRAF/IDL to Python Gotchas

There are some important differences in syntax between IRAF, IDL, and Python that new users should keep in mind. For more in depth information about indexing and slicing `Numpy` arrays see [their indexing documentation here](https://docs.scipy.org/doc/numpy-1.13.0/reference/arrays.indexing.html).

### x versus y
When working with images (2-dimensional) arrays, IRAF and IDL both have the index order `[x, y]`.  In Python's `Numpy` package, the order is reversed, `[y, x]`.

### index 0
IRAF indexes begin at 1 whereas Python and IDL both index arrays both start at zero.  So to  pull out the first element of a 1-dimensional array you would use `array[0]`.  To pull out the lower left corner of a 2-dimensional array you would use `array[0,0]`.

### slicing
Slicing in IRAF and IDL is inclusive for the right side of the slice. In Python the right side of the slice is exclusive.  For example, if you end a slice with the 4th index, `array[0:4]`, the fourth index element (actually the 5th element in the array since index begins at 0) will **not** be included in the slice.

### matplotlib origin
The default origin location for `matplotlib` plots (a common Python plotting library) will be in the upper-left.  To change this to the lower left (common for images) you can use the `origin=lower` parameter in the `imshow` call as follows: `plt.imshow(..., origin='lower')`.

<a id='links'></a>

# Links and Resources

### Astropy

**Main user documentation page:** http://docs.astropy.org/en/stable/

**Main FITS page:** http://docs.astropy.org/en/stable/io/fits/index.html

**Tutorials:** http://www.astropy.org/astropy-tutorials/

### Scipy and Numpy
**Main documentation pages:** https://docs.scipy.org/doc/

**Numpy indexing guide:** https://docs.scipy.org/doc/numpy-1.13.0/reference/arrays.indexing.html

### CCDProc
**Main documentation page:** http://ccdproc.readthedocs.io/en/latest/

### Matplotlib
**Main documentation page:** https://matplotlib.org/

## Ginga
**Main documentation page:** http://ginga.readthedocs.io/en/latest/

### Astroconda
**Main documentation page:** http://astroconda.readthedocs.io/en/latest/